In [ ]:
import json

json_data = '''
[
  { "id": 1, "name": "Rahul Sharma", "age": 21, "city": "Bangalore" },
  { "id": 2, "name": "Priya Singh", "age": 22, "city": "Delhi" },
  { "id": 3, "name": "Aman Kumar", "age": 20, "city": "Hyderabad" }
]
'''

students = json.loads(json_data)

print("Stusents Records:")
for s in students:
  print(f"{s['id']} - {s['name']} ({s['city']}) -> Age {s['age']}")

Stusents Records:
1 - Rahul Sharma (Bangalore) -> Age 21
2 - Priya Singh (Delhi) -> Age 22
3 - Aman Kumar (Hyderabad) -> Age 20


In [ ]:
import json

json_data = '''

[

  { "id": 1, "name": "Rahul Sharma", "age": 21, "city": "Bangalore" },

  { "id": 2, "name": "Priya Singh", "age": 22, "city": "Delhi" }

]

'''

students = json.loads(json_data)

new_student= {"id": 3, "name": "Aman Kumar" ,"age": 20,"city": "Hyderabad"}

students.append(new_student)

for s in students:
  if s["id"] ==1:
    s["city"] =="Pune"

updated_json = json.dumps(students, indent=2)

print("Updated JSON Data:\n",updated_json)

Updated JSON Data:
 [
  {
    "id": 1,
    "name": "Rahul Sharma",
    "age": 21,
    "city": "Bangalore"
  },
  {
    "id": 2,
    "name": "Priya Singh",
    "age": 22,
    "city": "Delhi"
  },
  {
    "id": 3,
    "name": "Aman Kumar",
    "age": 20,
    "city": "Hyderabad"
  }
]


In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Employee-Analysis").getOrCreate()


In [ ]:
import io
csv_data = """id,name,department,salary
1,Rahul Sharma,IT,55000
2,Priya Singh,HR,60000
3,Aman Kumar,Finance,48000
4,Sneha Reddy,Marketing,52000
5,Arjun Mehta,IT,75000
6,Divya Nair,Finance,67000
"""

with open("employees.csv","w") as f:
  f.write(csv_data)



In [ ]:
df = spark.read.csv("employees.csv",header=True, inferSchema=True)
df.show()

+---+------------+----------+------+
| id|        name|department|salary|
+---+------------+----------+------+
|  1|Rahul Sharma|        IT| 55000|
|  2| Priya Singh|        HR| 60000|
|  3|  Aman Kumar|   Finance| 48000|
|  4| Sneha Reddy| Marketing| 52000|
|  5| Arjun Mehta|        IT| 75000|
|  6|  Divya Nair|   Finance| 67000|
+---+------------+----------+------+



**Transformation**
# 📝 Key Points about Transformations

* **Lazy Execution**:
  Spark doesn’t run transformations right away. Instead, it builds a **logical plan** (a DAG – Directed Acyclic Graph).
  The computation only runs when an **action** (like `.show()` or `.count()`) is called.

* **Return Type**:
  A transformation always returns a **new DataFrame or RDD**. It does **not modify the existing one**.

* **Two Types of Transformations**:

  1. **Narrow Transformations** → Each input partition contributes to only one output partition.
     (e.g., `map()`, `filter()`, `select()`)
  2. **Wide Transformations** → Data is shuffled across partitions.
     (e.g., `groupBy()`, `join()`)

---

In [ ]:
df.select("name","salary").show()

df.filter(df["salary"] > 60000).show()

df.orderBy(df["salary"].desc()).show()

+------------+------+
|        name|salary|
+------------+------+
|Rahul Sharma| 55000|
| Priya Singh| 60000|
|  Aman Kumar| 48000|
| Sneha Reddy| 52000|
| Arjun Mehta| 75000|
|  Divya Nair| 67000|
+------------+------+

+---+-----------+----------+------+
| id|       name|department|salary|
+---+-----------+----------+------+
|  5|Arjun Mehta|        IT| 75000|
|  6| Divya Nair|   Finance| 67000|
+---+-----------+----------+------+

+---+------------+----------+------+
| id|        name|department|salary|
+---+------------+----------+------+
|  5| Arjun Mehta|        IT| 75000|
|  6|  Divya Nair|   Finance| 67000|
|  2| Priya Singh|        HR| 60000|
|  1|Rahul Sharma|        IT| 55000|
|  4| Sneha Reddy| Marketing| 52000|
|  3|  Aman Kumar|   Finance| 48000|
+---+------------+----------+------+



**Aggregation**


What is Aggregation?

* An operation that **groups data** and applies a **summary function** (like sum, avg, count, min, max).
* Used to answer questions like:

  * *“What is the average salary per department?”*
  * *“How many employees are in each department?”*
  * *“What is the highest salary in Finance?”*

---

In [ ]:
df.groupBy("department").avg("salary").show()

df.groupBy("department").max("salary").show()

df.groupBy("department").count().show()

+----------+-----------+
|department|avg(salary)|
+----------+-----------+
|        HR|    60000.0|
|   Finance|    57500.0|
| Marketing|    52000.0|
|        IT|    65000.0|
+----------+-----------+

+----------+-----------+
|department|max(salary)|
+----------+-----------+
|        HR|      60000|
|   Finance|      67000|
| Marketing|      52000|
|        IT|      75000|
+----------+-----------+

+----------+-----+
|department|count|
+----------+-----+
|        HR|    1|
|   Finance|    2|
| Marketing|    1|
|        IT|    2|
+----------+-----+



In [ ]:
df.createReplaceTempView("employees")
spark.sql("SELECT department, AVG (salary) as avg_salary FROM employees GROUP BY department").show()

AttributeError: 'DataFrame' object has no attribute 'createReplaceTempView'